In [5]:
%display latex

# Radiative Transfer

## Outline
We are using a backwards-ray tracing algorithm, so the basic steps we follow are:
1. We set up a screen (divided into a grid of pixels) at the observer's location. This is where the photons emitted from the accretion disk are captured.
2. We launch a null geodesic from each pixel of said screen towards the black hole. 
3. We find the intersection points of the geodesic with whatever accretion disk we've set up <br> and keep the part of the geodesic that lies withing this disk.
4. For each point of the numerical solution of the geodesic that is in the disk, we calculate the emmited radiation by considering the local conditions. We need to calculate:
    1. The fluid's 4-velocity. We do so assuming Keplerian rotation for the disk
    2. The local emissivity
    3. The intensity of the radiation
5. We transform the radiation to the observer's frame accounting for the Doppler shift and gravitational redshift


        
    


Assume we are following a single light ray. The portion of it that lies in the disk is denoted as **geodesic_sol** and it is a list of lists of the form $[\lambda,t,r,\theta,\phi]$. We calculate its 4-velocity at every point (iterating through the solution), calculating its numerical derivatives: 

In [4]:
def calc_4_velocity4(geodesic_sol):
    result = []
    for i in range(2, len(geodesic_sol)-2):
        r = geodesic_sol[i][1]
        vr = (-geodesic_sol[i+2][2] + 8*geodesic_sol[i+1][2] - 8*geodesic_sol[i-1][2] + geodesic_sol[i-2][2]) / (12*(geodesic_sol[i+1][0] - geodesic_sol[i][0]))
        vtheta = (-geodesic_sol[i+2][3] + 8*geodesic_sol[i+1][3] - 8*geodesic_sol[i-1][3] + geodesic_sol[i-2][3]) / (12*(geodesic_sol[i+1][0] - geodesic_sol[i][0]))
        vphi = (-geodesic_sol[i+2][4] + 8*geodesic_sol[i+1][4] - 8*geodesic_sol[i-1][4] + geodesic_sol[i-2][4]) / (12*(geodesic_sol[i+1][0] - geodesic_sol[i][0]))
        vt = (-geodesic_sol[i+2][1] + 8*geodesic_sol[i+1][1] - 8*geodesic_sol[i-1][1] + geodesic_sol[i-2][1]) / (12*(geodesic_sol[i+1][0] - geodesic_sol[i][0]))
        result.append((geodesic_sol[i][0],[geodesic_sol[i][1],geodesic_sol[i][2],geodesic_sol[i][3],geodesic_sol[i][4],vt, vr, vtheta, vphi]))
    return result

To make things simpler, **calc_4_velocity4** returns a list of the form
$[\lambda,[t,r,\theta,\phi,\dot{t},\dot{r},\dot{\theta},\dot{\phi}]$, so for every point of the geodesic we have everything we need in a single list.



Now that we have the light ray and its 4-velocity, let's be a little bit clearer about the process before moving on to the code.

At every point, there is an interaction of the disk with the photon. This interaction is encoded in the quantity $-k_a u^a$ that is the energy of the photon in the fluid's local rest frame. Here, $k_a$ is the photon's 4-momentum and $u^a$ is the fluid's 4-velocity. We essentially take this energy to be that of an emitted photon at that point of the disk that then travels to meet our screen. To account for the redshift of the photon's energy, we'll use the redshift factor <br>
<br>
$$\gamma^{-1}=\frac{\nu_{em}}{\nu_{obs}}=\frac{-k_a u^a}{E_{0}}=\frac{E_{em}}{E_{0}}$$

where in this equation, $E_0$ is the energy we use to launch the light ray, $E_0=1$, and serves as a reference. To calculate the energy of the photon when it reaches our screen, $E_{obs}$, we use $E_{obs}=\gamma E_{em}$. 

Having set a local rest-frame emissivity $j0$, we can then calculate the intensity of every photon through $$I=\gamma^{-1}\left(\frac{j_0}{E^3_{em}}\right),$$ assuming that the absorption coefficient is $a_0=0$.

In conclusion, for every light ray that travels through the accretion disk, there will be a photon of a specific energy and intensity emitted in every point. We will use all of these photons to create a histogram of intensity and the ratio $\frac{E_{obs}}{E_{em}}$.

<div class="alert alert-block alert-danger">
<b>Point for clarification:</b> We use $\frac{E_{em}}{E_{0}}$ to define the inverse of the redshift factor and then use this factor to find the observed energy at our screen. "Which of these $\gamma$s" should we use in the Intensity function? In the code below, I use the original one.
</div>

## Calculation of Intensity and Ratio

The function below, is called for every point of a geodesic.<br> **Note:** Assuming Keplerian motion for the disk, its azimuthial velocity at every point is given by:<br><br>$$v_{\phi}=\sqrt{\frac{m}{ (r^3 + a \sqrt{(m r)})}}$$

In [8]:
def intensity_at_point(points_4_velocity, E0, a_value, m_value):
    t, r, theta, phi, vt, vr, vtheta, vphi = points_4_velocity
    z = r * np.cos(theta)
    density = density_at_point(r, z,boundaries) #simple 1/r^n for now#
    
    #We manually set up the Kerr metric
    
    rho2 = r**2 + (a_value * np.cos(theta))**2
    Delta = r**2 - 2*m_value*r + a_value**2
    g = np.array([
        [-1 + 2*m_value*r / rho2, 0, 0, -2*a_value*m_value*r * np.sin(theta)**2 / rho2],
        [0, rho2 / Delta, 0, 0],
        [0, 0, rho2, 0],
        [-2*a_value*m_value*r * np.sin(theta)**2 / rho2, 0, 0, (r**2 + a_value**2 + 2*a_value*m_value*r * np.sin(theta)**2 / rho2) * np.sin(theta)**2]
    ])

    
    #Use the metric to calculate the photon's 4-momentum from its 4-velocity
    k_alpha = np.dot(g, np.array([vt, vr, vtheta, vphi]))

    
    #Calculate the fluids 4-velocity.
    
    #Assuming Keplerian motion, its azimuthial velocity will be given by:
    vphi_fluid = calculate_azimuthal_velocity(r, theta, a_value, m_value)
    
    
    #temporary fluid 4-velocity.
    u_beta_candidate = np.array([1, 0, 0, vphi_fluid])
    
    
    #set up a dot product with the metric to normalize the 4-velocity
    dot_product = np.dot(u_beta_candidate, np.dot(g, u_beta_candidate))
    

    #4-velocity Normalization
    vt_fluid = np.sqrt(-1 / dot_product)
    vphi_fluid = vphi_fluid*np.sqrt(-1 / dot_product)
    

    #Final 4-velocity of the Fluid
    u_beta = np.array([vt_fluid, 0, 0, vphi_fluid])
    

    #Energy of the photon in the fluid's rest frame
    k_alpha_u_alpha = np.dot(k_alpha, u_beta)
    E_em=-k_alpha_u_alpha
    
    
    #redshift factor
    gamma_inv = E_em / E0
    gamma = 1 / gamma_inv


    #energy that reaches the observer's screen
    E_obs = E_em*gamma 


    #optical depth if any
    optical_depth = 0

    #absorption if any
    alpha0 = 0  

    # update optical depth
    optical_depth += gamma_inv * alpha0

    #local rest-frame emissivity j0
    j0 = density  # Here, we use density as a proxy



    #Intensity
    intensity = gamma_inv * (j0 / E_em**3) * np.exp(-optical_depth)

    ratio=E_obs/E_em


    return intensity, ratio